## Load Data

In [1]:
function loadData(filename)
    rawData = readlines(filename)
    iniPolymer = split(rawData[1],"")
    rules = Dict{String,String}()
    for i in 3:length(rawData)
        splitLine = split(rawData[i])
        rules[splitLine[1]] = splitLine[3]
    end
    return iniPolymer,rules
end

loadData (generic function with 1 method)

In [2]:
iniPolymerTest,rulesTest = loadData("insertionRulesTest.txt")

(SubString{String}["N", "N", "C", "B"], Dict("HB" => "C","CH" => "B","BC" => "B","HC" => "B","NN" => "C","HH" => "N","BH" => "H","CC" => "N","NB" => "B","BB" => "N"…))

In [3]:
iniPolymer,rules = loadData("insertionRules.txt")

(SubString{String}["P", "H", "V", "C", "V", "B", "F", "H", "C", "V", "P", "F", "K", "B", "N", "H", "K", "N", "B", "O"], Dict("FN" => "V","PO" => "V","CC" => "N","BC" => "B","KF" => "B","BV" => "N","OP" => "O","PF" => "V","NB" => "F","VN" => "S"…))

## Part 1

In [4]:
function insertElements(origPolymer,rules)
    lenOrigPolymer = length(origPolymer)
    newPolymer = Array{String}(undef,0)
    for i in 1:lenOrigPolymer-1
        push!(newPolymer,origPolymer[i])
        if join(origPolymer[i:i+1]) in keys(rules)
            push!(newPolymer,rules[join(origPolymer[i:i+1])])
        end
    end
    push!(newPolymer,origPolymer[end])
    return newPolymer
end

function buildPolymer(origPolymer,rules,niter)
    polymer = origPolymer
    for i in 1:niter
        polymer = insertElements(polymer,rules)
    end
    return polymer
end

function analyzePolymer(polymer)
    uniqueElements = Set(polymer)
    numElements = zeros(Int,length(uniqueElements))
    i = 0
    for c in uniqueElements
        i+=1
        numElements[i] = count(x ->x == c,polymer) 
    end
    return sort(numElements)
end

analyzePolymer (generic function with 1 method)

In [5]:
finPolymerTest = buildPolymer(iniPolymerTest,rulesTest,10);

In [6]:
nElements = analyzePolymer(finPolymerTest)
nElements[end]-nElements[1]

1588

In [7]:
finPolymer = buildPolymer(iniPolymer,rules,10);
nElements = analyzePolymer(finPolymer)
nElements[end]-nElements[1]

3555

## Part 2

In [42]:
function updatePolymerCount(polymerPieces::Dict{String,Int},rules::Dict{String,String})
    newPolymerPieces = Dict(key=>0 for key in keys(polymerPieces))
    for (piece,n) in polymerPieces
        if n>0
            newPolymerPieces[string(piece[1],rules[piece])] += n
            newPolymerPieces[string(rules[piece],piece[2])] += n
        end
    end
    return newPolymerPieces
end

function initPolymerCount(origPolymer::String,rules::Dict{String,String})
    polymerPieces = Dict(key=>0 for key in keys(rules))
    lenPolymer = length(origPolymer)
    for i in 1:lenPolymer-1
        polymerPieces[join(origPolymer[i:i+1])] += 1
    end
    return polymerPieces   
end

function constructPolymer(origPolymer::String,rules::Dict{String,String},niter::Int)
    polymer = initPolymerCount(origPolymer,rules)
    for i in 1:niter
        polymer = updatePolymerCount(polymer,rules)
    end
    return polymer
end

function analyzePolymerNew(polymer,lastElement)
    uniqueElements = Set(join(keys(polymer)))
    elementCount = Dict(x => 0 for x in uniqueElements)
    println(uniqueElements)
    elementCount[lastElement[1]] +=1 
    for (key,n) in polymer
        elementCount[key[1]] += n
    end
    
    nelements = Array{Int}(undef,length(uniqueElements))
    i = 1
    for (key,n) in elementCount
        nelements[i] = n
        i+=1
    end
    sort!(nelements)
    return elementCount,(nelements[end]-nelements[1])
end

analyzePolymerNew (generic function with 1 method)

In [43]:
finPolymerTest = constructPolymer(join(iniPolymerTest),rulesTest,40)
nElements = analyzePolymerNew(finPolymerTest,iniPolymerTest[end])

Set(['H', 'B', 'C', 'N'])


(Dict('C' => 6597635301,'H' => 3849876073,'N' => 1096047802353,'B' => 2192039569602), 2188189693529)

In [44]:
finPolymer = constructPolymer(join(iniPolymer),rules,40)
nElements = analyzePolymerNew(finPolymer,iniPolymer[end])

Set(['B', 'P', 'K', 'H', 'V', 'O', 'C', 'F', 'N', 'S'])


(Dict('P' => 3178628005225,'K' => 927295863022,'C' => 755642661164,'F' => 2014715170153,'N' => 1322198384438,'H' => 1378718959755,'V' => 2501671713686,'S' => 1866024773848,'O' => 5195084704903,'B' => 1750740691551…), 4439442043739)